This notebook is used to prepare the dataset that will be used for the job recommender chatbot. The initial dataset of LinkedIn Job Postings - 2023 (job_postings.csv) from https://www.kaggle.com/datasets/arshkon/linkedin-job-postings has numerous industries, job titles, locations (ALL IN USA), etc. We will limit the number of industries and job titles just for the purpose of simplified chatbot implementation.

# **Import Needed Libraries**

In [ ]:
# Import needed libraries
import numpy as np # for linear algebra
import pandas as pd # for loading data from csv file and data processing
import matplotlib.pyplot as plt # to plot figures
import re

# **Upload Dataset**

In [ ]:
job_to_industry = pd.read_csv("/content/job_industries.csv")

In [ ]:
industry_name = pd.read_csv("/content/industries.csv")

In [ ]:
df= pd.read_csv("/content/job_postings.csv")
df

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped
0,3757940104,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",...,NaN,Entry level,NaN,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",...,NaN,NaN,NaN,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420
2,3757938019,474443.0,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",...,NaN,NaN,Bachelor's Degree in Mechanical Engineering pr...,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644
3,3757938018,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",...,NaN,Entry level,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,...,NaN,Mid-Senior level,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33241,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,NaN,NaN,NaN,NaN,Full-time,"Santa Clarita, CA",...,NaN,NaN,NaN,1.692830e+12,NaN,0,FULL_TIME,NaN,NaN,1
33242,108965123,NaN,Office Administrative Assistant,"A fast-fashion wholesaler, is looking for a fu...",NaN,NaN,NaN,NaN,Full-time,"New York, NY",...,NaN,NaN,NaN,1.699040e+12,NaN,0,FULL_TIME,NaN,NaN,1699044401
33243,102339515,52132271.0,Franchise Owner,DuctVentz is a dryer and A/C – heat vent clean...,NaN,NaN,NaN,NaN,Full-time,Greater Boston,...,NaN,NaN,NaN,1.699050e+12,NaN,0,FULL_TIME,NaN,NaN,1699063495
33244,85008768,NaN,Licensed Insurance Agent,While many industries were hurt by the last fe...,52000.0,NaN,45760.0,YEARLY,Full-time,"Chico, CA",...,NaN,NaN,NaN,1.692750e+12,NaN,1,FULL_TIME,USD,BASE_SALARY,1


In [ ]:
df.columns

Index(['job_id', 'company_id', 'title', 'description', 'max_salary',
       'med_salary', 'min_salary', 'pay_period', 'formatted_work_type',
       'location', 'applies', 'original_listed_time', 'remote_allowed',
       'views', 'job_posting_url', 'application_url', 'application_type',
       'expiry', 'closed_time', 'formatted_experience_level', 'skills_desc',
       'listed_time', 'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'scraped'],
      dtype='object')

# **Drop unnecessary columns**

We will drop the columns which are not relevant to the job recommendation task and which don't provide useful information for the clustering and classification of jobs.

In [ ]:
df = df.drop(['company_id', 'max_salary',
       'med_salary', 'min_salary', 'pay_period', 'applies', 'original_listed_time', 'remote_allowed',
       'views','application_url', 'application_type',
       'expiry', 'closed_time',
       'listed_time', 'posting_domain', 'sponsored', 'currency',
       'compensation_type', 'scraped', 'skills_desc', 'work_type'],axis='columns')
df

,job_id,title,description,formatted_work_type,location,job_posting_url,formatted_experience_level
0,3757940104,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,Full-time,"Little River, SC",https://www.linkedin.com/jobs/view/3757940104/...,Entry level
1,3757940025,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,Full-time,"Beaver Dam, WI",https://www.linkedin.com/jobs/view/3757940025/...,NaN
2,3757938019,"Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,Full-time,"Bessemer, AL",https://www.linkedin.com/jobs/view/3757938019/...,NaN
3,3757938018,Cook,descriptionTitle\n\n Looking for a great oppor...,Full-time,"Aliso Viejo, CA",https://www.linkedin.com/jobs/view/3757938018/...,Entry level
4,3757937095,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",Full-time,United States,https://www.linkedin.com/jobs/view/3757937095/...,Mid-Senior level
...,...,...,...,...,...,...,...
33241,133114754,Sales Manager,Are you a dynamic and creative marketing profe...,Full-time,"Santa Clarita, CA",https://www.linkedin.com/jobs/view/133114754/?...,NaN
33242,108965123,Office Administrative Assistant,"A fast-fashion wholesaler, is looking for a fu...",Full-time,"New York, NY",https://www.linkedin.com/jobs/view/108965123/?...,NaN
33243,102339515,Franchise Owner,DuctVentz is a dryer and A/C – heat vent clean...,Full-time,Greater Boston,https://www.linkedin.com/jobs/view/102339515/?...,NaN
33244,85008768,Licensed Insurance Agent,While many industries were hurt by the last fe...,Full-time,"Chico, CA",https://www.linkedin.com/jobs/view/85008768/?t...,NaN


In [ ]:
df.columns

Index(['job_id', 'title', 'description', 'formatted_work_type', 'location',
       'job_posting_url', 'formatted_experience_level'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33246 entries, 0 to 33245
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   job_id                      33246 non-null  int64 
 1   title                       33246 non-null  object
 2   description                 33245 non-null  object
 3   formatted_work_type         33246 non-null  object
 4   location                    33246 non-null  object
 5   job_posting_url             33246 non-null  object
 6   formatted_experience_level  24065 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.8+ MB


# **Drop Missing Values**

In [ ]:
# Drop entries with missing values for the narrative attribute
df= df.dropna()
# Rearrange indices to be of the form 0, 1, …, n - 1.
df= df.reset_index(drop=True)
df

,job_id,title,description,formatted_work_type,location,job_posting_url,formatted_experience_level
0,3757940104,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,Full-time,"Little River, SC",https://www.linkedin.com/jobs/view/3757940104/...,Entry level
1,3757938018,Cook,descriptionTitle\n\n Looking for a great oppor...,Full-time,"Aliso Viejo, CA",https://www.linkedin.com/jobs/view/3757938018/...,Entry level
2,3757937095,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",Full-time,United States,https://www.linkedin.com/jobs/view/3757937095/...,Mid-Senior level
3,3757937037,Territory Manager - New Haven,"Location: Remote, CT, United States of America...",Full-time,United States,https://www.linkedin.com/jobs/view/3757937037/...,Mid-Senior level
4,3757937004,Auto Body Techncian,Company: Gerber Collision & Glass\n\nWELCOME T...,Full-time,"Daytona Beach, FL",https://www.linkedin.com/jobs/view/3757937004/...,Entry level
...,...,...,...,...,...,...,...
24060,2148434077,Transporter - Full Time Evenings 3pm-11pm; Rot...,Service / Ancillary Staff --> Housekeeping\nPa...,Full-time,"Patchogue, NY",https://www.linkedin.com/jobs/view/2148434077/...,Entry level
24061,2148434033,Contract Bilingual Recruiter,"Location: Phoenix, AZ 85006---- IN OFFICE JOB ...",Contract,"Phoenix, AZ",https://www.linkedin.com/jobs/view/2148434033/...,Associate
24062,2148434032,Tax Analyst,RaceTrac Company Overview\n\nJob Description:\...,Full-time,"Atlanta, GA",https://www.linkedin.com/jobs/view/2148434032/...,Entry level
24063,2148434019,"Virtual Sales Associate Account Manager, Women...",Job Description Summary \n\nAs the Virtual Sal...,Full-time,"Connecticut, United States",https://www.linkedin.com/jobs/view/2148434019/...,Entry level


# **Map industry_id to industry names**

In [ ]:
industry_id = []
for job in df['job_id']:
  industry_id.append(job_to_industry[job_to_industry['job_id']==job]['industry_id'].values[0])

In [ ]:
industry_names = []
for id in industry_id:
  industry_names.append(industry_name[industry_name['industry_id']==id]['industry_name'].values[0])

In [ ]:
df['industry']= industry_names

**NOTE: THIS INDUSTRY LABEL WILL NOT BE USED LATER ON. INDUSTRIES WILL BE REDEFINED BASED ON CHOSEN JOB TITLES.**

# **Map formatted_experience_level values to Beginner, Intermediate, and Senior**

In [ ]:
df.loc[(df['formatted_experience_level']=='Entry level')|(df['formatted_experience_level']=='Internship'),'formatted_experience_level']='Beginner'
df.loc[(df['formatted_experience_level']=='Associate'),'formatted_experience_level']='Intermediate'
df.loc[(df['formatted_experience_level']=='Mid-Senior level')|(df['formatted_experience_level']=='Director')|(df['formatted_experience_level']=='Executive'),'formatted_experience_level']='Senior'

# **Drop jobs with unknown work type**

In [ ]:
df = df.drop(df[df['formatted_work_type']=='Other'].index)
df= df.reset_index(drop=True)
df

,job_id,title,description,formatted_work_type,location,job_posting_url,formatted_experience_level,industry
0,3757940104,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,Full-time,"Little River, SC",https://www.linkedin.com/jobs/view/3757940104/...,Beginner,Medical Equipment Manufacturing
1,3757938018,Cook,descriptionTitle\n\n Looking for a great oppor...,Full-time,"Aliso Viejo, CA",https://www.linkedin.com/jobs/view/3757938018/...,Beginner,Non-profit Organizations
2,3757937095,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",Full-time,United States,https://www.linkedin.com/jobs/view/3757937095/...,Senior,Retail
3,3757937037,Territory Manager - New Haven,"Location: Remote, CT, United States of America...",Full-time,United States,https://www.linkedin.com/jobs/view/3757937037/...,Senior,Medical Equipment Manufacturing
4,3757937004,Auto Body Techncian,Company: Gerber Collision & Glass\n\nWELCOME T...,Full-time,"Daytona Beach, FL",https://www.linkedin.com/jobs/view/3757937004/...,Beginner,Automotive
...,...,...,...,...,...,...,...,...
24005,2148434077,Transporter - Full Time Evenings 3pm-11pm; Rot...,Service / Ancillary Staff --> Housekeeping\nPa...,Full-time,"Patchogue, NY",https://www.linkedin.com/jobs/view/2148434077/...,Beginner,Hospitals and Health Care
24006,2148434033,Contract Bilingual Recruiter,"Location: Phoenix, AZ 85006---- IN OFFICE JOB ...",Contract,"Phoenix, AZ",https://www.linkedin.com/jobs/view/2148434033/...,Intermediate,Accounting
24007,2148434032,Tax Analyst,RaceTrac Company Overview\n\nJob Description:\...,Full-time,"Atlanta, GA",https://www.linkedin.com/jobs/view/2148434032/...,Beginner,Retail
24008,2148434019,"Virtual Sales Associate Account Manager, Women...",Job Description Summary \n\nAs the Virtual Sal...,Full-time,"Connecticut, United States",https://www.linkedin.com/jobs/view/2148434019/...,Beginner,Hospitals and Health Care


# **Drop jobs that don't mention the specific US state**

In [ ]:
df = df.drop(df[df['location']=='United States'].index)
df= df.reset_index(drop=True)
df

,job_id,title,description,formatted_work_type,location,job_posting_url,formatted_experience_level,industry
0,3757940104,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,Full-time,"Little River, SC",https://www.linkedin.com/jobs/view/3757940104/...,Beginner,Medical Equipment Manufacturing
1,3757938018,Cook,descriptionTitle\n\n Looking for a great oppor...,Full-time,"Aliso Viejo, CA",https://www.linkedin.com/jobs/view/3757938018/...,Beginner,Non-profit Organizations
2,3757937004,Auto Body Techncian,Company: Gerber Collision & Glass\n\nWELCOME T...,Full-time,"Daytona Beach, FL",https://www.linkedin.com/jobs/view/3757937004/...,Beginner,Automotive
3,3757936167,"ACME D8- Asst Store Director (ASD) Sussex, NJ",The First Assistant Store Director is actively...,Full-time,"Sussex, NJ",https://www.linkedin.com/jobs/view/3757936167/...,Senior,Retail
4,3757936097,Dishwasher,"descriptionTitle\n\n $2,000 Sign-on Bonus Guar...",Full-time,"Aliso Viejo, CA",https://www.linkedin.com/jobs/view/3757936097/...,Beginner,Non-profit Organizations
...,...,...,...,...,...,...,...,...
22598,2148434077,Transporter - Full Time Evenings 3pm-11pm; Rot...,Service / Ancillary Staff --> Housekeeping\nPa...,Full-time,"Patchogue, NY",https://www.linkedin.com/jobs/view/2148434077/...,Beginner,Hospitals and Health Care
22599,2148434033,Contract Bilingual Recruiter,"Location: Phoenix, AZ 85006---- IN OFFICE JOB ...",Contract,"Phoenix, AZ",https://www.linkedin.com/jobs/view/2148434033/...,Intermediate,Accounting
22600,2148434032,Tax Analyst,RaceTrac Company Overview\n\nJob Description:\...,Full-time,"Atlanta, GA",https://www.linkedin.com/jobs/view/2148434032/...,Beginner,Retail
22601,2148434019,"Virtual Sales Associate Account Manager, Women...",Job Description Summary \n\nAs the Virtual Sal...,Full-time,"Connecticut, United States",https://www.linkedin.com/jobs/view/2148434019/...,Beginner,Hospitals and Health Care


# **Rename similar job titles**

We renamed similar job for easier chatbot implementation so that it aligns with Dialogflow.

In [ ]:
df.loc[df['title'].apply(lambda x: ('Data Engineer' in x)|('Data Developer' in x)),'title']='Data Engineer'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Data Scientist' in x)|('Data Analy' in x)|('Data Architect' in x)),'title']='Data Scientist'

In [ ]:
df.loc[df['title'].apply(lambda x: 'Database' in x),'title']='Database Management'

In [ ]:
df.loc[df['title'].apply(lambda x: 'Developer' in x),'title']='Developer'

In [ ]:
df.loc[df['title'].apply(lambda x: ('UI' in x)|('UX' in x)),'title']='UI/UX Designer'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Software Engineer' in x)),'title']='Software Engineer'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Nurse' in x)),'title']='Nurse'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Nursing Assistant' in x)),'title']='Nursing Assistant'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Veterinarian' in x)),'title']='Veterinarian'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Veterinary Assistant' in x)),'title']='Veterinary Assistant'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Veterinary Technician' in x)),'title']='Veterinary Technician'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Dental Assistant' in x)),'title']='Dental Assistant'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Dental Hygienist' in x)),'title']='Dental Hygienist'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Dentist' in x)),'title']='Dentist'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Account Executive' in x)),'title']='Account Executive'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Portfolio Manag' in x)),'title']='Portfolio Manager'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Financial Analyst' in x)),'title']='Financial Analyst'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Banker' in x)),'title']='Banker'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Accountant' in x)),'title']='Accountant'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Tax' in x)),'title']='Tax Professional'

In [ ]:
df.loc[df['title'].apply(lambda x: ('Retirement' in x)),'title']='Retirement Specialist'

# **Choose a couple of job titles for each of the 3 industries: IT, Healthcare, Finance**

In [ ]:
IT_list = ['Data Engineer','Data Scientist','Database Management','Developer','UI/UX Designer','Software Engineer']
Healthcare_list = ['Nurse','Nursing Assistant','Veterinarian','Veterinary Assistant','Veterinary Technician','Dental Assistant','Dental Hygienist','Dentist']
Finance_list = ['Account Executive','Portfolio Manager','Financial Analyst','Retirement Specialist','Banker','Accountant','Tax Professional']

In [ ]:
all_titles = IT_list + Healthcare_list + Finance_list

In [ ]:
df = df[df['title'].isin(all_titles)]

In [ ]:
df= df.reset_index(drop=True)
df

,job_id,title,description,formatted_work_type,location,job_posting_url,formatted_experience_level,industry
0,3757935025,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"St Clair Shores, Michigan, United States",https://www.linkedin.com/jobs/view/3757935025/...,Senior,Veterinary Services
1,3757934327,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"Denver, CO",https://www.linkedin.com/jobs/view/3757934327/...,Senior,Veterinary Services
2,3757933458,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"Santa Rosa, CA",https://www.linkedin.com/jobs/view/3757933458/...,Senior,Veterinary Services
3,3757932826,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"Pittsburgh, PA",https://www.linkedin.com/jobs/view/3757932826/...,Senior,Veterinary Services
4,3757931806,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"Syracuse, NY",https://www.linkedin.com/jobs/view/3757931806/...,Senior,Veterinary Services
...,...,...,...,...,...,...,...,...
2649,3684441939,Accountant,Titus Talent Strategies has partnered with SAR...,Full-time,"Madison, WI",https://www.linkedin.com/jobs/view/3684441939/...,Beginner,Leasing Non-residential Real Estate
2650,3682818140,Developer,Carefully read the JD before applying!Job Desc...,Contract,"McKinney, TX",https://www.linkedin.com/jobs/view/3682818140/...,Senior,Insurance
2651,3663878663,Accountant,Bring YOUR energy to Alliant Energy!\n\nAt All...,Full-time,"Madison, WI",https://www.linkedin.com/jobs/view/3663878663/...,Beginner,Utilities
2652,3584068782,Account Executive,Position SummaryThe Account Executive role foc...,Full-time,San Francisco Bay Area,https://www.linkedin.com/jobs/view/3584068782/...,Senior,Legal Services


# **Redefine industry column**

In [ ]:
df.loc[df['title'].isin(IT_list),'industry']='IT'

In [ ]:
df.loc[df['title'].isin(Healthcare_list),'industry']='Healthcare'

In [ ]:
df.loc[df['title'].isin(Finance_list),'industry']='Finance'

In [ ]:
df

,job_id,title,description,formatted_work_type,location,job_posting_url,formatted_experience_level,industry
0,3757935025,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"St Clair Shores, Michigan, United States",https://www.linkedin.com/jobs/view/3757935025/...,Senior,Healthcare
1,3757934327,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"Denver, CO",https://www.linkedin.com/jobs/view/3757934327/...,Senior,Healthcare
2,3757933458,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"Santa Rosa, CA",https://www.linkedin.com/jobs/view/3757933458/...,Senior,Healthcare
3,3757932826,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"Pittsburgh, PA",https://www.linkedin.com/jobs/view/3757932826/...,Senior,Healthcare
4,3757931806,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,"Syracuse, NY",https://www.linkedin.com/jobs/view/3757931806/...,Senior,Healthcare
...,...,...,...,...,...,...,...,...
2649,3684441939,Accountant,Titus Talent Strategies has partnered with SAR...,Full-time,"Madison, WI",https://www.linkedin.com/jobs/view/3684441939/...,Beginner,Finance
2650,3682818140,Developer,Carefully read the JD before applying!Job Desc...,Contract,"McKinney, TX",https://www.linkedin.com/jobs/view/3682818140/...,Senior,IT
2651,3663878663,Accountant,Bring YOUR energy to Alliant Energy!\n\nAt All...,Full-time,"Madison, WI",https://www.linkedin.com/jobs/view/3663878663/...,Beginner,Finance
2652,3584068782,Account Executive,Position SummaryThe Account Executive role foc...,Full-time,San Francisco Bay Area,https://www.linkedin.com/jobs/view/3584068782/...,Senior,Finance


# **Convert abbreviated state names in location column to full state name**

In [ ]:
abbreviated_states = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

state_names = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California",
    "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas",
    "Kentucky", "Louisiana", "Maine", "Maryland",
    "Massachusetts", "Michigan", "Minnesota", "Mississippi",
    "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire",
    "New Jersey", "New Mexico", "New York", "North Carolina",
    "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
    "Rhode Island", "South Carolina", "South Dakota", "Tennessee",
    "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]

for state, abb in zip(state_names, abbreviated_states):
  df.loc[df['location'].apply(lambda x: (state in x)|(abb in x)),'location']=state

In [ ]:
df=df[df['location'].isin(state_names)]
df= df.reset_index(drop=True)
df

,job_id,title,description,formatted_work_type,location,job_posting_url,formatted_experience_level,industry
0,3757935025,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,Michigan,https://www.linkedin.com/jobs/view/3757935025/...,Senior,Healthcare
1,3757934327,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,Colorado,https://www.linkedin.com/jobs/view/3757934327/...,Senior,Healthcare
2,3757933458,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,California,https://www.linkedin.com/jobs/view/3757933458/...,Senior,Healthcare
3,3757932826,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,Pennsylvania,https://www.linkedin.com/jobs/view/3757932826/...,Senior,Healthcare
4,3757931806,Veterinarian,We are looking to hire an Associate Veterinari...,Full-time,New York,https://www.linkedin.com/jobs/view/3757931806/...,Senior,Healthcare
...,...,...,...,...,...,...,...,...
2574,3685418185,Software Engineer,"Who we are:Founded in 2017, Gatik is the leade...",Full-time,California,https://www.linkedin.com/jobs/view/3685418185/...,Senior,IT
2575,3684441939,Accountant,Titus Talent Strategies has partnered with SAR...,Full-time,Wisconsin,https://www.linkedin.com/jobs/view/3684441939/...,Beginner,Finance
2576,3682818140,Developer,Carefully read the JD before applying!Job Desc...,Contract,Texas,https://www.linkedin.com/jobs/view/3682818140/...,Senior,IT
2577,3663878663,Accountant,Bring YOUR energy to Alliant Energy!\n\nAt All...,Full-time,Wisconsin,https://www.linkedin.com/jobs/view/3663878663/...,Beginner,Finance


# **Exploration of obtained dataset after preprocessing**

In [ ]:
df.columns

Index(['job_id', 'title', 'description', 'formatted_work_type', 'location',
       'job_posting_url', 'formatted_experience_level', 'industry'],
      dtype='object')

## **Distribution of industry in obtained dataset**

In [ ]:
df['industry'].value_counts()

Healthcare    1008
Finance        812
IT             759
Name: industry, dtype: int64

## **Distribution of job titles in obtained data set**

In [ ]:
df['title'].value_counts()

Nurse                    817
Accountant               311
Developer                300
Software Engineer        205
Tax Professional         180
Account Executive        147
Data Scientist           124
Financial Analyst        101
Banker                    53
Data Engineer             53
Dental Hygienist          45
Veterinarian              42
UI/UX Designer            42
Nursing Assistant         40
Database Management       35
Dental Assistant          24
Veterinary Technician     17
Veterinary Assistant      16
Portfolio Manager         13
Dentist                    7
Retirement Specialist      7
Name: title, dtype: int64

## **Distribution of location in the obtained dataset**

In [ ]:
df['location'].value_counts()

California        300
Texas             220
New York          212
Florida           173
Washington        122
Illinois          108
Massachusetts     105
Georgia            98
Virginia           97
North Carolina     91
Ohio               81
Arizona            76
Pennsylvania       69
Michigan           67
Maryland           66
Colorado           63
New Jersey         63
Tennessee          42
Connecticut        41
Missouri           40
South Carolina     36
Wisconsin          32
Indiana            31
Oregon             29
Kansas             29
Kentucky           27
Minnesota          25
Iowa               21
Utah               21
Oklahoma           20
Idaho              18
Nebraska           16
Maine              16
Alabama            14
Nevada             13
Delaware           13
New Hampshire      11
Louisiana          10
Arkansas           10
Alaska              8
New Mexico          8
West Virginia       7
Rhode Island        6
North Dakota        5
Hawaii              5
Montana   

## **Distribution of experience level in the obtained dataset**

In [ ]:
df['formatted_experience_level'].value_counts()

Senior          1685
Beginner         616
Intermediate     278
Name: formatted_experience_level, dtype: int64

## **Distribution of work type in the obtained dataset**

In [ ]:
df['formatted_work_type'].value_counts()

Full-time     1986
Contract       297
Part-time      278
Internship      10
Temporary        8
Name: formatted_work_type, dtype: int64

# **Renaming columns**

In [ ]:
df.rename(columns = {"title":"job_title"}, inplace = True)
df.rename(columns = {"description":"job_desc"}, inplace = True)
df.rename(columns = {"formatted_work_type":"work_type"}, inplace = True)
df.rename(columns = {"formatted_experience_level":"experience_level"}, inplace = True)

# **Rearranging columns**

In [ ]:
df.columns

Index(['job_id', 'job_title', 'job_desc', 'work_type', 'location',
       'job_posting_url', 'experience_level', 'industry'],
      dtype='object')

In [ ]:
columns= ['job_id','job_title','job_desc','experience_level','work_type','location','industry','job_posting_url' ]

In [ ]:
df=df[columns]

# **Saving to csv file**

In [ ]:
df.to_csv('linkedin_jobs.csv',index=False)